In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict

from zlu_functions import *
from nel_functions import *
from helper import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
pickle_in=open('hedge_dfs_subset.pickle','rb')
another_dct=pickle.load(pickle_in)

In [3]:
another_dct = trimDictonary(another_dct,[]) 
del_dfs_dic = delete_duplicated(another_dct) 
del_dfs_dic2 = delete_subset(another_dct)
for k in another_dct.keys():
    for i in range(len(another_dct[k])):
        df=another_dct[k][i]
        df=set_col_headers(df) 
        if isinstance(df, pd.DataFrame):
            df=del_subset_col(df)
            df=remove_rows_for_total_value(df)
            df=remove_cols_for_fair_value(df)
            df=dfs_need_transpose(df)
            df=row_header_to_col(df)
            another_dct[k][i]=df

---

In [4]:
len_dict = defaultdict(int)
for key in another_dct.keys():
    len_dict[key] = len(another_dct[key])

In [5]:
len_dict

defaultdict(int,
            {'FST20030321': 5,
             'FST20040315': 5,
             'FST20050315': 5,
             'FST20060316': 4,
             'FST20070228': 3,
             'FST20080228': 4,
             'FST20090302': 4,
             'FST20100301': 4,
             'FST20110224': 4,
             'FST20120222': 3,
             'FST20130222': 3,
             'FST20140226': 3,
             'GDP20060315': 1,
             'GDP20070314': 2,
             'GDP20080313': 3,
             'GDP20090227': 2,
             'GDP20100226': 1,
             'GDP20110222': 1,
             'GDP20120224': 2,
             'GDP20130222': 2,
             'GDP20140221': 2,
             'GDP20150302': 1,
             'ISRL20090323': 1,
             'ISRL20100312': 3,
             'ISRL20110311': 4,
             'MRO20040309': 1,
             'MRO20060306': 1,
             'MRO20080229': 1,
             'MRO20090227': 1,
             'MRO20100226': 2,
             'MRO20130222': 2,
             'MRO20

---

In [6]:
# Vertically Stacked Subtable Example
len(another_dct['GDP20090227'])
another_dct['GDP20090227'][0]
another_dct['GDP20090227'][1]

2

,Collars (NYMEX),DailyVolume,TotalVolume,AverageFloor/Cap,product_type
2,1Q 2009,20000,1800000,$8.75|$13.10,natural gas (mmbtu)
3,2Q 2009,20000,1820000,$8.75|$13.10,natural gas (mmbtu)
4,3Q 2009,20000,1840000,$8.75|$13.10,natural gas (mmbtu)
5,4Q 2009,20000,1840000,$8.75|$13.10,natural gas (mmbtu)
7,Swaps (NYMEX),,,Average Price,
9,1Q 2009,20000,1800000,$8.83,natural gas (mmbtu)
10,2Q 2009,20000,1820000,$8.83,natural gas (mmbtu)
11,3Q 2009,20000,1840000,$8.83,natural gas (mmbtu)
12,4Q 2009,20000,1840000,$8.83,natural gas (mmbtu)
14,Swaps (TexOk),,,Price (1),


,Collars (NYMEX),DailyVolume,TotalVolume,Floor/CapAverage Price,product_type
2,1Q 2009,20000,1800000,$8.75|$13.10,natural gas (mmbtu)
3,2Q 2009,20000,1820000,$8.75|$13.10,natural gas (mmbtu)
4,3Q 2009,20000,1840000,$8.75|$13.10,natural gas (mmbtu)
5,4Q 2009,20000,1840000,$8.75|$13.10,natural gas (mmbtu)
7,Swaps (NYMEX),,,Average Price,
9,1Q 2009,20000,1800000,$8.83,natural gas (mmbtu)
10,2Q 2009,20000,1820000,$8.83,natural gas (mmbtu)
11,3Q 2009,20000,1840000,$8.83,natural gas (mmbtu)
12,4Q 2009,20000,1840000,$8.83,natural gas (mmbtu)
14,Swaps (TexOk),,,Field Price (1),


In [7]:
def get_vertically_stacked_subtables(dictionary):
    '''
    We want to find dataframes that satisfy one of the following conditions, to be flagged as subtables:
        Condition A: has one or more rows that contain only strings
        Condition B: volume columns contain any empty strings
    This function outputs a dictionary indicating tickers and respective indices for subtables.
    
    dictionary: dictionary with file_header as keys and list of dataframes as values
    '''
    
    # Initialize an empty defaultdict to store subtables
    flagged_dict = defaultdict(set)
    
    for k in dictionary.keys():
        target_list = dictionary[k]
        for index in range(len(target_list)):
            # Make sure it's a dataframe and it's not empty
            if isinstance(target_list[index], pd.DataFrame) & (not target_list[index].empty):
                df = target_list[index]
                Cond_A = False

            ### Condition A
                # Check if the dataframe has rows that have no numbers to be found (i.e. all strings)
                for x in range(df.shape[0]):
                    boolean_row = []
                    for y in range(df.shape[1]):
                        # Skip cells that are empty
                        if df.iloc[x, y] != '':
                            # Removes parentheses and everything within it, for each cell
                            df.iloc[x, y] = re.sub(r'\([^()]*\)', '', df.iloc[x, y])
                            # Iterate every cells within a row to see if number exists
                            if re.findall('[\d]', df.iloc[x, y]) == []:
                                boolean_row.append(True)
                            else: boolean_row.append(False)
                    # If there exists a row where no numbers are to be found, flag that dataframe as subtable
                    if np.array(boolean_row).all():
                        # Output the potentially subtable to flagged_dict
                        flagged_dict[k].add(index)
                        Cond_A = True

            ### Condition B
                # Check to see if Condition A is satisfied
                if Cond_A:
                    # Check to see if there are any volume containing columns
                    if (df.columns.astype(str).str.contains('Volume', case=False)).any():
                        # If so, see if there are any missing strings within those columns
                        if (df[df.columns[df.columns.astype(str).str.contains('Volume')]] == '').any().any():
                            flagged_dict[k].add(index) 
                            
    # return the subtable dictionary for bookkeeping and input for vertically_stacked_subtable_cleaner()                       
    return flagged_dict

In [8]:
# I have manually gone through all dataframes one by one, and here are the problematic ones (total: 11)

# 'GDP20060315':[0]
# 'GDP20080313':[0]
# 'GDP20090227':[0,1]
# 'GDP20100226':[0]
# 'GDP20110222':[0]
# 'GDP20120224':[0]
# 'MRO20100226':[0,1]
# 'PLLL20080220':[1]
# 'PLLL20090223':[0]

In [9]:
flagged_dict = get_vertically_stacked_subtables(another_dct)
flagged_dict

defaultdict(set,
            {'FST20130222': {1},
             'FST20140226': {1},
             'GDP20060315': {0},
             'GDP20080313': {0},
             'GDP20090227': {0, 1},
             'GDP20100226': {0},
             'GDP20110222': {0},
             'GDP20120224': {0},
             'MRO20100226': {0, 1},
             'PLLL20080220': {1},
             'PLLL20090223': {0}})

In [10]:
# a generator so that we can just click next to view individual dataframes
df_gen = ((another_dct[ticker][index]) for ticker, indices in flagged_dict.items() for index in indices)

In [11]:
next(df_gen)

,",Commodity Options,,Underlying TermGas Swaptions",",Commodity Options,,Gas Swaptions",",Commodity Options,,Option ExpirationGas Swaptions",",Commodity Options,Natural Gas (NYMEX HH),Underlying BbtuPer Day",",Commodity Options,Natural Gas (NYMEX HH),UnderlyingHedged Priceper MMBtu",",Commodity Options,Oil (NYMEX WTI),UnderlyingBarrelsPer Day",",Commodity Options,Oil (NYMEX WTI),UnderlyingHedged Priceper Bbl"
5,Calendar 2014,,December 2013,30,$4.50,,
6,Calendar 2014,,December 2013,10,4.51,,
7,Oil Swaptions,Oil Swaptions,Oil Swaptions,,,,
8,Calendar 2014,,December 2013,,,2000,110.00
9,Calendar 2014,,December 2013,,,1000,109.00
10,Calendar 2014,,December 2013,,,2000,100.00
11,Calendar 2015,,December 2014,,,3000,100.00


In [12]:
def vertically_stacked_subtable_cleaner(dictionary, flagged_dict):
    '''
    This function does two things:
    1) Appends the splitted subtables to the list where the subtables are contained in
    2) Deletes the original subtables
    
    dictionary: dictionary with file_header as keys and list of dataframes as values
    flagged_dict: output from get_vertically_stacked_subtables()
    '''
    
### Append subtables to original list under respective tickers
    for ticker, indices in flagged_dict.items():
        for index in indices:
            df_original = dictionary[ticker][index]
            # Removes parentheses and everything within it, for each cell
            df_temp = df_original.applymap(lambda x: re.sub(r'\([^()]*\)', "", x))
            # Assuming we don't have to split more than fifteen times per dataframe
            for _ in range(15):
                for x_idx in range(df_original.shape[0]):
                    # Find the first row index to split on
                    if not any(df_temp.iloc[x_idx].str.contains('\d')): 
                        break
                # If it's not a subtable anymore
                if (x_idx + 1) == df_original.shape[0]:
                    break
                df_split1 = df_original.iloc[:x_idx]
                df_split2 = df_original.iloc[x_idx:]
                # Keep the original columns as series for further usage
                columns_temp = df_split2.columns
                # Clear the columns to use set_col_header function
                df_split2.columns = ["" for x in columns_temp]
                df_split2 = set_col_headers(df_split2)
                # Use original columns if empty OR if the new would_have_been column names are contained in columns_temp, or else, stick with the new ones
                df_split2.columns = [columns_temp[k] if (re.search('[\w]',v) == None or v in columns_temp[k]) else v for k,v in enumerate(df_split2.columns)]
                dictionary[ticker].append(df_split1)
                df_original = df_split2
            # Grab the final piece of the dataframe 
            dictionary[ticker].append(df_split2)

### Delete the original subtables
    for k in flagged_dict.keys():
        target_list = dictionary[k]
        to_be_del = list(flagged_dict[k])
        to_be_del.sort(reverse=True)
        for index in to_be_del:
            del target_list[index]

In [13]:
vertically_stacked_subtable_cleaner(another_dct, flagged_dict)

In [14]:
len(another_dct['GDP20090227'])
another_dct['GDP20090227'][0]
another_dct['GDP20090227'][1]
another_dct['GDP20090227'][2]
another_dct['GDP20090227'][3]
another_dct['GDP20090227'][4]
another_dct['GDP20090227'][5]

6

,Collars (NYMEX),DailyVolume,TotalVolume,AverageFloor/Cap,product_type
2,1Q 2009,20000,1800000,$8.75|$13.10,natural gas
3,2Q 2009,20000,1820000,$8.75|$13.10,natural gas
4,3Q 2009,20000,1840000,$8.75|$13.10,natural gas
5,4Q 2009,20000,1840000,$8.75|$13.10,natural gas


,Swaps,DailyVolume,TotalVolume,Average Price,product_type
9,1Q 2009,20000,1800000,$8.83,natural gas
10,2Q 2009,20000,1820000,$8.83,natural gas
11,3Q 2009,20000,1840000,$8.83,natural gas
12,4Q 2009,20000,1840000,$8.83,natural gas


,Swaps,DailyVolume,TotalVolume,Price,product_type
16,1Q 2009,20000,1800000,$7.87,natural gas
17,2Q 2009,20000,1820000,$7.87,natural gas
18,3Q 2009,20000,1840000,$7.87,natural gas
19,4Q 2009,20000,1840000,$7.87,natural gas


,Collars (NYMEX),DailyVolume,TotalVolume,Floor/CapAverage Price,product_type
2,1Q 2009,20000,1800000,$8.75|$13.10,natural gas
3,2Q 2009,20000,1820000,$8.75|$13.10,natural gas
4,3Q 2009,20000,1840000,$8.75|$13.10,natural gas
5,4Q 2009,20000,1840000,$8.75|$13.10,natural gas


,Swaps,DailyVolume,TotalVolume,Floor/CapAverage Price,product_type
9,1Q 2009,20000,1800000,$8.83,natural gas
10,2Q 2009,20000,1820000,$8.83,natural gas
11,3Q 2009,20000,1840000,$8.83,natural gas
12,4Q 2009,20000,1840000,$8.83,natural gas


,Swaps,DailyVolume,TotalVolume,Field Price,product_type
16,1Q 2009,20000,1800000,$7.87,natural gas
17,2Q 2009,20000,1820000,$7.87,natural gas
18,3Q 2009,20000,1840000,$7.87,natural gas
19,4Q 2009,20000,1840000,$7.87,natural gas


In [15]:
del_dfs_dic3 = delete_duplicated(another_dct)
del_dfs_dic3

defaultdict(list,
            {'GDP20090227': [5, 4, 3], 'MRO20130222': [1], 'XCO20060331': [5]})